In [1]:
# Atenção: como o arquivo é imenso, ele foi redimensionado para um arquivo de 9 linhas apenas para testar os comandos antes de aplicar no dataset completo.

# in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839
# uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304 0
# uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0" 304 0
# uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/MOSAIC-logosmall.gif HTTP/1.0" 304 0
# uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/USA-logosmall.gif HTTP/1.0" 304 0
# ix-esc-ca2-07.ix.netcom.com - - [01/Aug/1995:00:00:09 -0400] "GET /images/launch-logo.gif HTTP/1.0" 200 1713
# uplherc.upl.com - - [01/Aug/1995:00:00:10 -0400] "GET /images/WORLD-logosmall.gif HTTP/1.0" 304 0
# slppp6.intermind.net - - [01/Aug/1995:00:00:10 -0400] "GET /history/skylab/skylab.html HTTP/1.0" 200 1687
# piweba4y.prodigy.com - - [01/Aug/1995:00:00:10 -0400] "GET /images/launchmedium.gif HTTP/1.0" 200 11853

# 1. Número de hosts únicos.
# 2. O total de erros 404.
# 3. Os 5 URLs que mais causaram erro 404.
# 4. Quantidade de erros 404 por dia.
# 5. O total de bytes retornados.

In [2]:
# Create a RDD file
rdd = sc.textFile('/FileStore/tables/teste')

In [3]:
import re

def log_parser(line):

  # This method parse the log
  # The log format(standard) is Apache Common Log Format (CLF): https://httpd.apache.org/docs/1.3/logs.html#common
  # Example:
  # in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839

  # Compile regex pattern for better performance
  regex_pattern = re.compile('^(\S+) (\S) (\S) \[(\d{2}/\w{3}/\d{4}:\d{2}:\d{2}:\d{2}) (\+|-\d{4})\] \"(\S+)\s*(\S+) (\w+)/(\d{1}.\d{1})\" (\d{3}) (\d+)')
  
  match = regex_pattern.match(line)

  if match:
      parsed = [match.groups()[0],
                match.groups()[3],
                match.groups()[4],
                match.groups()[5],
                match.groups()[6],
                match.groups()[7],
                match.groups()[8],
                match.groups()[9],
                match.groups()[10]]
      return parsed
  else:
    print(line)

In [4]:
# TODO
# Testar se o parse fica melhor chamando o método no map, ou se fica melhor no select aplicando o regexp_extract
# regex_pattern = r'^(\S+) (\S) (\S) \[(\d{2}/\w{3}/\d{4}:\d{2}:\d{2}:\d{2}) (\+|-\d{4})\] \"(\S+)\s*(\S+) (\w+)/(\d{1}.\d{1})\" (\d{3}) (\d+)'
# rdd.select(regexp_extract(col('host'), r'(regex_pattern)', 1),
#             regexp_extract(col('datetime'), r'(regex_pattern)', 2).collect()

# Parse every line
parsed_rdd = rdd.map(lambda line: log_parser(line))

In [5]:
# Convert RDD to DataFrame
parsed_df = parsed_rdd.toDF(['host', 'datetime', 'timezone', 'request_method', 'url', 'protocol', 'protocol_version', 'status_code', 'bytes_transferred'])

In [6]:
display(parsed_df)

host,datetime,timezone,request_method,url,protocol,protocol_version,status_code,bytes_transferred
in24.inetnebr.com,01/Aug/1995:00:00:01,-0400,GET,/shuttle/missions/sts-68/news/sts-68-mcc-05.txt,HTTP,1.0,200,1839
uplherc.upl.com,01/Aug/1995:00:00:07,-0400,GET,/,HTTP,1.0,304,0
uplherc.upl.com,01/Aug/1995:00:00:08,-0400,GET,/images/ksclogo-medium.gif,HTTP,1.0,304,0
uplherc.upl.com,01/Aug/1995:00:00:08,-0400,GET,/images/MOSAIC-logosmall.gif,HTTP,1.0,304,0
uplherc.upl.com,01/Aug/1995:00:00:08,-0400,GET,/images/USA-logosmall.gif,HTTP,1.0,304,0
ix-esc-ca2-07.ix.netcom.com,01/Aug/1995:00:00:09,-0400,GET,/images/launch-logo.gif,HTTP,1.0,200,1713
uplherc.upl.com,01/Aug/1995:00:00:10,-0400,GET,/images/WORLD-logosmall.gif,HTTP,1.0,304,0
slppp6.intermind.net,01/Aug/1995:00:00:10,-0400,GET,/history/skylab/skylab.html,HTTP,1.0,200,1687
piweba4y.prodigy.com,01/Aug/1995:00:00:10,-0400,GET,/images/launchmedium.gif,HTTP,1.0,200,11853


In [7]:
# 1. Número de hosts únicos.
parsed_df.select('host').distinct().count()

Out[7]: 5

In [8]:
# 2. O total de erros 404.
parsed_df.filter(parsed_df['status_code'] == '404').count()

Out[8]: 0

In [9]:
# 3. Os 5 URLs que mais causaram erro 404.
# WIP
# Está fazendo a soma agrupado por host, porém ainda não ordena para obter os 5 primeiros
parsed_df.filter(parsed_df['status_code'] == '304').groupBy('host').count().show()

+---------------+-----+
 host|count|
+---------------+-----+
uplherc.upl.com| 5|
+---------------+-----+

In [10]:
# 4. Quantidade de erros 404 por dia.
# TODO

In [11]:
# 5. O total de bytes retornados.
# TODO
# Gerar o DataFrame com schema e colocar a coluna 'bytes_transferred' como int

parsed_df.select(parsed_df['bytes_transferred'].cast('int')).groupBy().sum().show()

+----------------------+
sum(bytes_transferred)|
+----------------------+
 17092|
+----------------------+